# Model comparison

In [11]:
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
import numpy as np
from scipy.io import loadmat
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error, r2_score

from models import Residual5, DNN
import tensorflow
from random import seed

seed(85679)
tensorflow.random.set_seed(85679)

## PLS

In [12]:
data = loadmat('./dataset/mango/mango_dm_split.mat')
min_value, max_value = data['min_y'][-1][-1], data['max_y'][-1][-1]
retransform = lambda x: x * (max_value - min_value)

In [13]:
x_train, y_train, x_test, y_test = data['x_train'], data['y_train'], data['x_test'], data['y_test']
print(f"shape of data:\n"
      f"x_train: {x_train.shape}, y_train: {y_train.shape},\n"
      f"x_test: {x_test.shape}, y_test: {y_test.shape}")

shape of data:
x_train: (8183, 102), y_train: (8183, 1),
x_test: (3508, 102), y_test: (3508, 1)


In [14]:
pls = PLSRegression(n_components=90)
svr = SVR(kernel="rbf", degree=30, gamma="scale")
mlp = MLPRegressor(hidden_layer_sizes=(60, 50, ))
pls = pls.fit(x_train, y_train.ravel())
svr = svr.fit(x_train, y_train.ravel())
mlp = mlp.fit(x_train, y_train.ravel())

models = {'PLS': pls, "SVR": svr, "MLP": mlp}
results = {model_name: model.predict(x_test).reshape((-1, )) for model_name, model in models.items()}
for model_name, model_result in results.items():
      rmse = np.sqrt(mean_squared_error(y_test, model_result))
      print(model_name, "RMSE: ", rmse)
      print(model_name, "Dry matter content error", retransform(rmse))
      print(model_name, "R^2: ", r2_score(y_test, model_result))

PLS RMSE:  0.05722520296881164
PLS Dry matter content error 0.8648183977750965
PLS R^2:  0.8793937498230511
SVR RMSE:  0.1139650997574326
SVR Dry matter content error 1.7223025845485895
SVR R^2:  0.5216575965112935
MLP RMSE:  0.15231159655301899
MLP Dry matter content error 2.3018157046177476
MLP R^2:  0.14560032918931198


In [15]:
epoch, batch_size = 1024, 64
resnet = Residual5(input_shape=(1, 102))
dnn = DNN(input_shape=(1, 102))
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=12, shuffle=True)
x_train, x_val, x_test = x_train[:, np.newaxis, :], x_val[:, np.newaxis, :], x_test[:, np.newaxis, :]

In [16]:
_ = resnet.fit(x_train, y_train, x_val, y_val, batch_size=batch_size, epoch=epoch)

Epoch 1/1024
90/90 [==============================] - 1s 2ms/step - loss: 0.0167 - val_loss: 0.0340 - lr: 0.0025
Epoch 2/1024
90/90 [==============================] - 0s 1ms/step - loss: 0.0071 - val_loss: 0.0264 - lr: 0.0025
Epoch 3/1024
90/90 [==============================] - 0s 943us/step - loss: 0.0052 - val_loss: 0.0701 - lr: 0.0025
Epoch 4/1024
90/90 [==============================] - 0s 1ms/step - loss: 0.0048 - val_loss: 0.0245 - lr: 0.0025
Epoch 5/1024
90/90 [==============================] - 0s 1ms/step - loss: 0.0043 - val_loss: 0.0236 - lr: 0.0025
Epoch 6/1024
90/90 [==============================] - 0s 848us/step - loss: 0.0044 - val_loss: 0.1782 - lr: 0.0025
Epoch 7/1024
90/90 [==============================] - 0s 846us/step - loss: 0.0045 - val_loss: 0.2228 - lr: 0.0025
Epoch 8/1024
90/90 [==============================] - 0s 822us/step - loss: 0.0039 - val_loss: 0.2283 - lr: 0.0025
Epoch 9/1024
90/90 [==============================] - 0s 845us/step - loss: 0.0043 - val

In [17]:
_ = dnn.fit(x_train, y_train, x_val, y_val, batch_size=batch_size, epoch=epoch)

Epoch 1/1024
90/90 [==============================] - 0s 1ms/step - loss: 0.0266 - val_loss: 0.0246 - lr: 0.0025
Epoch 2/1024
90/90 [==============================] - 0s 1ms/step - loss: 0.0240 - val_loss: 0.0249 - lr: 0.0025
Epoch 3/1024
90/90 [==============================] - 0s 1ms/step - loss: 0.0240 - val_loss: 0.0242 - lr: 0.0025
Epoch 4/1024
90/90 [==============================] - 0s 933us/step - loss: 0.0243 - val_loss: 0.0241 - lr: 0.0025
Epoch 5/1024
90/90 [==============================] - 0s 940us/step - loss: 0.0233 - val_loss: 0.0233 - lr: 0.0025
Epoch 6/1024
90/90 [==============================] - 0s 916us/step - loss: 0.0228 - val_loss: 0.0231 - lr: 0.0025
Epoch 7/1024
90/90 [==============================] - 0s 824us/step - loss: 0.0228 - val_loss: 0.0265 - lr: 0.0025
Epoch 8/1024
90/90 [==============================] - 0s 828us/step - loss: 0.0232 - val_loss: 0.0244 - lr: 0.0025
Epoch 9/1024
90/90 [==============================] - 0s 827us/step - loss: 0.0236 - v

In [21]:
resnet = load_model("checkpoints/res5.hdf5")
cnn = load_model("checkpoints/plain11.hdf5")
dnn = load_model("checkpoints/dnn.hdf5")

In [22]:
models = {'resnet': resnet, 'cnn': cnn, 'dnn': dnn}
results = {model_name: model.predict(x_test).reshape((-1, )) for model_name, model in models.items()}
for model_name, model_result in results.items():
      rmse = np.sqrt(mean_squared_error(y_test, model_result))
      print("="*20)
      print(model_name, "rmse : ", rmse)
      print(model_name, "Dry matter content error", retransform(rmse))
      print(model_name, "r^2 :", r2_score(y_test, model_result))

resnet rmse :  0.05082896493225598
resnet Dry matter content error 0.7681549690131755
resnet r^2 : 0.9048480565064634
cnn rmse :  0.05200769624271875
cnn Dry matter content error 0.7859685978067217
cnn r^2 : 0.9003837097594369
dnn rmse :  0.14165380273751216
dnn Dry matter content error 2.1407493266379727
dnn r^2 : 0.2609878018465487
